<a href="https://colab.research.google.com/github/ThapaVinay/Coding/blob/master/Copy_of_GNN_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch-geometric
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.1+cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 26.9 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=2818ee05e1b3bf469ca2a2a206ff99fd4020880fbdd053dd34dc502c31cf02c7
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built torch-geometric
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 62.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import networkx as nx
import torch
import matplotlib.pyplot as plt


In [3]:
import os
train_path = '/content/drive/MyDrive/train_data/'
test_path = '/content/drive/MyDrive/test_data/'
# files = os.listdir(path)[:3]    # only first three folders
files = os.listdir(train_path)
classes = {'1' :1}
files

['1']

In [4]:
def get_label(img_name):
  ind = img_name.index('.')
  a = ord(img_name[ind-1]) - ord('0')
  if(a % 2 == 0):
      return 0
  else:
      return 1

In [5]:
from PIL import Image
def read_image(img_path):
  img = Image.open(img_path)
  return img

In [6]:
def img_to_array(img):
  # print(img.size)  # width and height 
  img_array = np.array(img)
  img_array = img_array
  return img_array

In [22]:
# Add nodes to the graph
def img_to_graph(img_array):
  G = nx.Graph()
  rows, cols= img_array.shape
  for i in range(rows):
      for j in range(cols):
          pixel = (img_array[i][j], img_array[i][j], img_array[i][j])
          G.add_node((i, j), rgb = pixel)

  # Add edges between adjacent nodes
  for i in range(rows):
      for j in range(cols):
          if i < rows-1:
              G.add_edge((i, j), (i+1, j))
          if j < cols-1:
              G.add_edge((i, j), (i, j+1))      
  # plt.figure(figsize=(10,10))
  # pos = {n: n for n in G.nodes()}
  # nx.draw_networkx_nodes(G, pos, node_size=10, node_color=img.getdata())
  # nx.draw_networkx_edges(G, pos, alpha=0.2)
  # plt.axis('off')
  # plt.show()
  return G 

In [9]:

# def img_to_graph(img_array):
#   # create a new graph
#   G = nx.Graph()
#   height, width = img_array.shape
#   for x in range(height):
#       for y in range(width):
#           # node_label = str(x) + "," + str(y)  # label node with its x,y coordinates
#           pixel_value = img_array[x, y]
#           G.add_node(pixel_value)


#   # connect adjacent pixels in the graph with edges
#   for x in range(height):
#       for y in range(width):
#           node1_label = img_array[x,y]
#           if x < height - 1:
#               node2_label = img_array[x+1,y]
#               G.add_edge(node1_label, node2_label)
#           if y < width - 1:
#               node3_label = img_array[x,y+1]
#               G.add_edge(node1_label, node3_label)

#   return G


In [10]:
# # adding node feature to the graph
# def add_node_features(G,img):
#   img = img.convert('RGB')
#   for node in G.nodes:
#     G.nodes[node]['rgb'] = np.array([node,node,node])

In [14]:
import torch
def edges_to_tensor(G):
  nodes = []
  for node in G.nodes():
    nodes.append(node)
  nodes
  # Compute the edges of the graph
  edges = []
  for i in range(len(nodes)):
      for j in range(i + 1, len(nodes)):
          if abs(nodes[i][0] - nodes[j][0]) <= 1 and abs(nodes[i][1] - nodes[j][1]) <= 1:
              edges.append((i, j))

  # Convert the edges to a tensor
  edge_index = torch.tensor(edges).t().contiguous()

  return edge_index

In [12]:
# converting the edges into tensor
# def edges_to_tensor(G):
#   edge_index = np.array(list(G.edges()))
#   # count = 0

#   # for i in range(len(edge_index)):
#   #   for j in range(len(edge_index[0])):
#   #     edge_index[i][j] = count
#   #     count = count + 1

#   edge_index = torch.from_numpy(edge_index).to(torch.long).t().contiguous()
#   return edge_index 

In [13]:

# Assuming the node features are stored as the 'rgb' attribute of each node
# converting attributes to tensor
def attributes_to_tensor(G):
  num_nodes = len(G.nodes())
  first_node = next(iter(G))   
  num_features = len(G.nodes[first_node]['rgb'])
  x = torch.zeros((num_nodes, num_features), dtype=torch.float)
  for i, node in enumerate(G.nodes()):
      x[i] = torch.tensor(G.nodes[node]['rgb'], dtype=torch.float)
  return x



In [19]:
def add_labels(G,l):
  # assign label to the graph
  G.graph['label'] = l
  
  # converting to tensor
  y = torch.tensor([G.graph['label']], dtype=torch.long)
  return y

In [1]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import torch_geometric

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(3, 12) # 3 input channels (for RGB), 16 output channels
        self.conv2 = GCNConv(12, 32) # 16 input channels, 32 output channels
        self.conv3 = GCNConv(32, 64) # 32 input channels, 64 output channels
        self.classifier = Linear(64, 2)
        
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = global_mean_pool(x, torch.zeros(x.size(0), dtype=torch.long))
        out = self.classifier(x)
        return out

# Create the GCN model and optimizer
model = GCN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model for a few epochs
for epoch in range(10):
    model.train()

    for cl in classes:
      train_new_path = train_path + cl
      for img_name in os.listdir(train_new_path):

        # functions are defined above
        img = read_image(train_new_path+"/"+img_name)   # to access eah image 
        img_array = img_to_array(img)
        G = img_to_graph(img_array)
        # add_node_features(G,img)

        edge_index = edges_to_tensor(G)
        x = attributes_to_tensor(G)
        y = add_labels(G,get_label(img_name))

        optimizer.zero_grad()
        out = model(x, edge_index)
        # loss = criterion(out, y.unsqueeze(1))
        loss = criterion(out,y)
        # loss = F.cross_entropy(out,y)
        loss.backward()
        optimizer.step()
    
        # Evaluate the model on the validation set
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for cl in classes:
              new_path = test_path + cl
              for img_name in os.listdir(new_path):
                # functions are defined above
                img = read_image(new_path+"/"+img_name)
                img_array = img_to_array(img)
                G = img_to_graph(img_array)
                # add_node_features(G,img)

                edge_index = edges_to_tensor(G)
                x = attributes_to_tensor(G)
                y = add_labels(G,get_label(img_name))

                out = model(x, edge_index)
                pred = out.argmax(dim=1)

                # plt.figure(figsize = (15,2))
                # plt.imshow(img)
                # plt.xlabel(pred)
                # plt.show()

                correct += (pred == y).sum().item()
                total += 1
        acc = correct / total
        print(f"Epoch {epoch+1}, val acc = {acc:.4f}")


ModuleNotFoundError: ignored